# Introduction

## About

This notebook gathers the code used for the miniproject "Bayesian Inverse Problems in Large Dimensions" of the Stochastic Simulation course. See the report for theoretical details.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# Custom imports
from utils import u, G, target_density
from mcmc import rwmh
from utils import dump_simulation_results, load_simulation_results

In [4]:
# Given data
y = np.array([0.5041, 0.8505, 1.2257, 1.4113])

In [5]:
# Noise standard deviation
sigma_noise = 0.04
# Chain length
N = int(1e4)

# Random Walk Metropolis

In this first approach, we use the proposal distribution

$$Q(\mathbf \theta, \cdot) = \mathcal N(\mathbf \theta, s^2 C), \quad C = \text{diag}(1, 2^{-2}, \ldots, D^{-2})$$

where $D$ is the dimension of the state space, being the number of Fourier coefficients used to approximate the log-permeability.

The goal is to generate chains of length $N=10^4$ with different values of $s$ and $D$. Remind that we're trying to probe the efficiency of MCMC for $D \to\infty$. We will use the `rwmh` (Random Walk Metropolis Hastings) utility function:

In [6]:
help(rwmh)

Help on function rwmh in module mcmc:

rwmh(ftilde: Callable, variances: numpy.ndarray, X0: numpy.ndarray, N: int, verbose: bool = True)
    Random Walk Metropolis Hastings, i.e. type of Markov Chain Monte Carlo in continuous state space with Gaussian proposal
    density centered at current state.
    For a multidimensional state space, the components of the proposal samples are independent, i.e. the covariance
    matrix is diagonal.
    :param ftilde: un-normalized target density being the Markov Chain invariant distribution (after normalization)
    :param variances: 1D array of variances for each component of the proposal distribution
    :param X0: starting point of the chain
    :param N: chain length
    :return:



In [7]:
def simulate(args):
    """Perform one simulation"""
    D, s = args
    print(f'D = {D}, s = {s}\t', end='')
    X, p_accept = rwmh(
        ftilde=lambda theta: target_density(theta, sigma_noise, y),
        variances=s**2 / np.arange(1, D+1)**2,
        X0=np.zeros(D),
        N=N
    )
    # Only return the first component
    return {
        'X': X[:, 0],
        'D': D,
        's': s,
        'p_accept': p_accept
    }

In [8]:
import multiprocessing as mp
import itertools

In [9]:
D = (100, )
s = (0.1, 0.3, 0.5)
pool = mp.Pool(mp.cpu_count() - 2)
data = pool.map(simulate, itertools.product(D, s))
pool.close()
dump_simulation_results(data, 'rwmh_1.json.gz')

D = 100, s = 0.3	D = 100, s = 0.1	D = 100, s = 0.5	took 21.1 s   acceptance rate 0.0867
took 21.3 s   acceptance rate 0.0003
took 21.8 s   acceptance rate 0.559


In [10]:
# data = []
# for D in (100,):
#     for s in (0.1, 0.3):
#         X, p_accept = rwmh(
#             ftilde=lambda theta: target_density(theta, sigma_noise, y),
#             variances=s**2 / np.arange(1, D+1)**2,
#             X0=np.zeros(D),
#             N=N
#         )
#         data.append({
#             'X': X,
#             'D': D,
#             's': s,
#             'p_accept': p_accept
#         })
#
# dump_simulation_results(data, 'data/rwmh_1.json.gz')